In [ ]:
site = lambda state,start,stop : [f"usa_{state}_{i:04}" for i in range(start,stop+1)]

# Construct a dict of shoreline names to site ids
sitesD = {
'Virginia Beach': site('VA',4,4),
## Rudee inlet (Jettied)
'Duck': site('VA',1,3) + site('NC',39,49),
## Oregon Inlet
# 'Rodanthe': site('NC',32,38),
# Connected at Cape Hatteras
'Hatteras': site('NC',30,38),#31),
## Hatteras Inlet
'Ocracoke Island': site('NC',28,29),
## Ocracoke Inlet
'Portsmouth Island': site('NC',26,27),
# Connected at New Drum Inlet
'Dump Island': site('NC',25,25),
# Connected at Drum Inlet
'Cape Lookout': site('NC',22,24), 
## Barden Inlet
'Shackleford Banks': site('NC',21,21),
## Beaufort Inlet
'Atlantic Beach': site('NC',19,20), # 19 is Emerald Isle
## Bogue Inlet
'Bear Island': site('NC',18,18), # River at Bear Inlet?
## Browns Inlet
'Camp Lejeune': site('NC',17,17),
## New River Inlet
'Topsail Beach': site('NC',14,16),
## New Topsail Inlet?
'Lea-Hulaff Island': site('NC',13,13),
## Rich Inlet
'Figure Eight Island': site('NC',12,12),
## Mason Inlet?
'Wrightsville Beach': site('NC',11,11),
## Masonboro Inlet
'Masonboro Island': site('NC',10,10),
## Carolina Beach Inlet
# 'Carolina Beach': site('NC',7,9),
# Connected at Cape Fear
# 'Bald Head Island': site('NC',6,6),
'Cape Fear': site('NC',6,9),
## Cape Fear River?
'Oak Island': site('NC',4,5),
# Lockwoods Folly Inlet
'Holden Beach': site('NC',3,3),
# Tubbs Inlet
'Ocean Isle Beach': site('NC',2,2),
# Shallotte Inlet - jetty near south spit
'Sunset Beach': site('NC',1,1),
# Little River Inlet - jettied
'Watles Island': site('SC',28,28),
# Hog Inlet
'Myrtle Beach': site('SC',25,27), # lots of storm drains onto beach
# Murrells Inlet
'Litchfield Beach': site('SC',24,24),
# Midway Inlet
'Pawleys Island': site('SC',23,23),
# Pawleys Inlet - jettied -spit elongates tidal river w/o bay, transects are wrong
'Debidue Island': site('SC',22,22),
# North Inlet
'North Island': site('SC',21,21),
# Winyah Bay?
}
allSites = [x for xs in sitesD.values() for x in xs]#site('SC',21,28)+site('NC',1,49)+site('VA',1,4)
slNamesD = {v:k for k in sitesD.keys() for v in sitesD[k]}

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess

def mylowess(targetTs,col,ts,window=4,it=3): # smoothing window in years
    fracNull = (len(col)-col.isnull().sum())/len(col)
        # seconds in a solar year
    frac = ((86400*365.24219*window)/(max(ts)-min(ts))) * fracNull
    return lowess(col,ts,frac,it,xvals=targetTs)

In [ ]:
import pandas as pd
# newDates = pd.to_datetime(pd.date_range(start='1984', end='2026', freq='ME'),utc=True)
targetDates = pd.to_datetime(pd.date_range(start='1984', end='2026', freq='YE-JUN'),utc=True)
# There isn't anything visibly consistent about shorelines sampled more than yearly
# Higher sampling than twice a year doesn't work for 1984
# targetDates = pd.to_datetime(pd.date_range(start='3-1984', end='2026', freq='6ME'),utc=True)
targetTs = [date.timestamp() for date in targetDates]

In [ ]:
save=True#False

### New Idea:
- pandas interpolate followed by pandas.rolling().mean()
- not sure if I need to interpolate
- what if I insert regularly spaced dates and interpolate those?

In [ ]:
from concurrent.futures import ProcessPoolExecutor
# from statsmodels.nonparametric.smoothers_lowess import lowess

#!wget 'https://zenodo.org/records/15626280/files/shoreline_data.zip?download=1'
# import zipfile
# with zipfile.ZipFile('shoreline_data.zip', 'r') as zip_ref:
#     zip_ref.extractall('shoreline_data')
# import os
# os.chdir('shoreline_data')

def getOffsets(siteID,smooth=False,returnAll=False):
    # print(siteID+' ')
    df = pd.read_csv(f"{siteID}/time_series_tidally_corrected.csv").iloc[:,1:]
    # ts = df['dates'].apply(lambda x: pd.Timestamp(x).timestamp())
    df['dates'] = pd.to_datetime(df['dates'])
    df = df.set_index('dates')
    if smooth!='lowess':
        df = df.reindex(df.index.union(targetDates)).sort_index()
        df = df.interpolate(method='time')
    if smooth=='roll':
        df = df.rolling(f"{0.5*365.24219}D",min_periods=1,center=True).mean()
    if returnAll:
        return df
    # TODO: break dates at replenishment discontinuities and fit each separately
    if smooth=='lowess':
        print(siteID+' ')
        ts = [t.timestamp() for t in df.index]
        siteOffsets = df.apply(func=lambda col : mylowess(targetTs,col,ts)) # Takes 6.2 seconds with custom frac
        # siteOffsets = df.apply(func=lambda col : lowess(col,ts,0.05,xvals=targetTs)) # Takes 6.0s; 0.08 is mean of custom frac for usa_NC_0030; ranges between 0.05 and 0.09
        siteOffsets.index = targetDates#.strftime('%Y-%m-%d')
        siteOffsets = siteOffsets.T
    else:
        siteOffsets = df.loc[targetDates].T
    siteOffsets['shoreline'] = slNamesD[siteID]
    return siteOffsets

In [ ]:
with ProcessPoolExecutor() as executor:
    offsets = executor.map(getOffsets, allSites,['roll']*len(allSites))
offsets = pd.concat(offsets)

In [ ]:
import geopandas as gpd

# load the transects
#!wget 'https://zenodo.org/records/15626280/files/US_East_transects.geojson?download=1'
transects = gpd.read_file('US_East_transects.geojson')
# transects = transects.to_crs(transects.estimate_utm_crs())
transects = transects[transects['site_id'].isin(allSites)].to_crs("EPSG:5070")
transects = transects.join(offsets,on='id')

In [ ]:
# reset line startpoints and offsets to make negative distances go away
# geopandas interpolate measures negative distances from endpoints instead of backwatds from startpoints
from shapely.geometry import LineString

def extend_line_start(geom):
    """Extend a 2-point line in both directions by a given distance."""
    p1,p2 = geom.coords
    dx,dy = [p2[i] - p1[i] for i in range(2)]
    new_p1 = (p1[0] - (dx/geom.length) * geom.length, p1[1] - (dy/geom.length) * geom.length)
    return LineString([new_p1, p2])

cols = offsets.columns[offsets.dtypes=='float64']
transects[cols] = transects[cols].add(transects.geometry.length, axis=0)
transects.geometry = [ extend_line_start(g) for g in transects.geometry]

In [ ]:
import gzip
def save_gj(gdf,name,gz=True):
    gdf.columns = gdf.columns.map(str)
    if gz:
        with gzip.open(name+'.geojson.gz', 'wt') as f:
            f.write(gdf.to_json())
    else:
        gdf.to_file(name+'.geojson')

In [ ]:
import json
def read_gjgz(filename):
    return gpd.GeoDataFrame.from_features(json.load(gzip.open(filename, 'rt'))['features'])

In [ ]:
if save:
    print('Saving...')
    save_gj(transects.copy(),'transects')
    print('Saved')

In [ ]:
from shapely.geometry import LineString

def interpolate_group(group):
    geoms = [group[1].geometry.interpolate(group[1][targetDate]) for targetDate in targetDates]
    # Remove empty points (where shoreline does not cross transect)
    geoms = [g[~(g.is_empty | g.isna())] for g in geoms]
    return group[0], [LineString(g) for g in geoms]

# # Testing for empty points on Cape Lookout
# import numpy as np
# print(offsets.iloc[np.where(offsets.isnull())].drop_duplicates())
# items =[f"{ss[:6]}_0022{ss[-5:]}" for ss in site('NC',74,82)]
# interpolate_group(transects.set_index('id').filter(items=items,axis=0))

In [ ]:
# slsD = dict(list(transects.groupby('shoreline').apply(interpolate_group,include_groups=False)))
with ProcessPoolExecutor() as executor:
    slsD = executor.map(interpolate_group,transects.groupby('shoreline'))
slsD = dict(list(slsD))

In [ ]:
tdD = dict(zip(targetDates.strftime('%Y-%m-%d'),zip(*slsD.values())))

gdfs = {targetDate: gpd.GeoDataFrame(crs = "EPSG:5070", data = 
    {
    'Name' : slsD.keys(),
    'site_id' : [sitesD[name] for name in slsD.keys()],
    'date' : targetDate,
    'geometry' : tdD[targetDate]
    })
     for targetDate in targetDates.strftime('%Y-%m-%d')}

In [ ]:
if save:
    for targetDate in targetDates.strftime('%Y-%m-%d'):
        gdfs[targetDate].to_file('shorelines.gpkg',layer=targetDate)

In [ ]:
import folium
m = folium.Map()
# m = transects.iloc[:,:9].explore(name='transects',show=False)
for targetDate in targetDates.strftime('%Y-%m-%d')[:4]:
    gdfs[targetDate].explore(m=m,name=targetDate)
for targetDate in targetDates.strftime('%Y-%m-%d')[4:]:
    gdfs[targetDate].explore(m=m,name=targetDate,show=False)
folium.LayerControl().add_to(m)
m